In [1]:
from profit_scanner.profit_scanner import ProfitScanner
from transaction.transaction_loader import TransactionLoader

In [2]:
#enable autoreload
%load_ext autoreload
%autoreload 2

In [3]:
owner_address = "0x1a869357c99bdf201D0378971FfE0b78DA5697E0"

transaction_loader = TransactionLoader()
normal_transactions, erc20_transactions, internal_transactions = transaction_loader.load_transactions(owner_address)
profit_scanner = ProfitScanner(owner_address, normal_transactions, erc20_transactions, internal_transactions)

In [4]:
res = profit_scanner.process()

In [5]:
res

[SwapTransactionResult(owner_address='0x1a869357c99bdf201D0378971FfE0b78DA5697E0', function_name='swapETHForExactTokens(uint256 amountOut, address[] path, address to, uint256 deadline)', token_name='Loopy', token_symbol='粉红小海狸', swap_hash='0xb2d19dc9ae49ff0a5a47a3095dd147d8f1a2e590dfbad3333ed6ac703dc211cd', block_number=17926445, timestamp=1692177479, router_address='0x7a250d5630b4cf539739df2c5dacb4c659f2488d', token_address='0x423c1c27e2946eec52795e93fad991f9e5394570', token_amount=148500000.0, eth_amount=0.06439607834953986, transaction_fee=0.003587699872487889, direction='buy'),
 SwapTransactionResult(owner_address='0x1a869357c99bdf201D0378971FfE0b78DA5697E0', function_name='swapETHForExactTokens(uint256 amountOut, address[] path, address to, uint256 deadline)', token_name='Secret Organisation of Shibarium', token_symbol='SOS', swap_hash='0xbe1619b1a9e6404698f9303d915d78b5ed7dd4ba40fdcdadade2673395a7d870', block_number=17926566, timestamp=1692178943, router_address='0x7a250d5630b4cf